In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def create_covariance_matrix(x_points, sigma=1.0, length_scale=1.0):

    n = len(x_points)
    C = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):

            distance = abs(x_points[i] - x_points[j])
            C[i, j] = sigma**2 * np.exp(-0.5 * (distance / length_scale)**2)
    
    return C

def kl_expansion(x_points, n_terms=None, sigma=1.0, length_scale=1.0, variance_threshold=0.95):

    C = create_covariance_matrix(x_points, sigma, length_scale)
    
    eigenvalues, eigenfunctions = np.linalg.eigh(C)
    
    idx = np.argsort(eigenvalues)[::-1]  
    eigenvalues = eigenvalues[idx]
    eigenfunctions = eigenfunctions[:, idx]
    
    if n_terms is None:
        total_variance = np.sum(eigenvalues)
        cumulative_variance = np.cumsum(eigenvalues) / total_variance
        n_terms = np.where(cumulative_variance >= variance_threshold)[0][0] + 1
    
    n_terms = min(n_terms, len(x_points))
    
    return eigenvalues[:n_terms], eigenfunctions[:, :n_terms]

def generate_random_field(x_points, mean=0.0, sigma=1.0, length_scale=1.0, 
                         n_terms=None, random_seed=None):

    if random_seed is not None:
        np.random.seed(random_seed)
    
    eigenvalues, eigenfunctions = kl_expansion(
        x_points, n_terms, sigma, length_scale
    )
    
    n_terms = len(eigenvalues)
    xi = np.random.randn(n_terms)  
    
    Y = mean + np.dot(eigenfunctions, np.sqrt(eigenvalues) * xi)
    
    return Y, eigenvalues, eigenfunctions

def generate_stiffness_field(x_points, mean_E=200e9, cv=0.2, length_scale=1.0, 
                            n_terms=None, random_seed=None):

    sigma_log = np.sqrt(np.log(1 + cv**2))

    mu_log = np.log(mean_E) - 0.5 * sigma_log**2

    Y_field, eigenvalues, eigenfunctions = generate_random_field(
        x_points, mean=mu_log, sigma=sigma_log, 
        length_scale=length_scale, n_terms=n_terms, random_seed=random_seed
    )

    E_field = np.exp(Y_field)
    
    return E_field, Y_field, eigenvalues, eigenfunctions

def validate_kl_generator():

    L = 5.0  
    n_points = 100  
    x_points = np.linspace(0, L, n_points)

    test_cases = [
        {'cv': 0.1, 'length_scale': 0.5, 'label': 'lc=0.5 cv=0.1'},
        {'cv': 0.2, 'length_scale': 1.0, 'label': 'lc=1.0 cv=0.2'},
        {'cv': 0.3, 'length_scale': 2.0, 'label': 'lc=2.0 cv=0.3'},
    ]

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    for idx, params in enumerate(test_cases):
        cv = params['cv']
        length_scale = params['length_scale']
        
        E_field, Y_field, eigenvalues, eigenfunctions = generate_stiffness_field(
            x_points, mean_E=200e9, cv=cv, length_scale=length_scale, random_seed=idx
        )
        
        ax1 = axes[0, idx]
        ax1.plot(x_points, E_field / 1e9, 'b-', linewidth=2)
        ax1.axhline(y=200, color='r', linestyle='--', alpha=0.5, label='Average')
        ax1.set_xlabel('Position (m)')
        ax1.set_ylabel('E (GPa)')
        ax1.set_title(f'Stiffness Field: {params["label"]}\nCV={cv}, l={length_scale}m')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        ax2 = axes[1, idx]
        ax2.plot(x_points, Y_field, 'g-', linewidth=2)
        ax2.axhline(y=np.log(200e9), color='r', linestyle='--', alpha=0.5, label='Average')
        ax2.set_xlabel('Position (m)')
        ax2.set_ylabel('log(E)')
        ax2.set_title(f'log(Sitffness field)')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
    
    plt.tight_layout()
    
    print("=== Statistical testing of KL expansion ===")
    for idx, params in enumerate(test_cases):
        cv = params['cv']
        length_scale = params['length_scale']
        
        n_samples = 500
        stiffness_samples = []
        
        for i in range(n_samples):
            E_field, _, _, _ = generate_stiffness_field(
                x_points, mean_E=200e9, cv=cv, length_scale=length_scale, 
                random_seed=i+1000 
            )
            stiffness_samples.append(E_field)
        
        stiffness_samples = np.array(stiffness_samples)
        
        mean_E = np.mean(stiffness_samples)
        std_E = np.std(stiffness_samples)
        actual_cv = std_E / mean_E
        
        print(f"TEST {idx+1}: {params['label']}")
        print(f"  Aim average: 200 GPa, real average: {mean_E/1e9:.2f} GPa")
        print(f"  Aim CV: {cv:.2f}, real CV: {actual_cv:.2f}")
        print(f"  error: {abs(actual_cv-cv)/cv*100:.1f}%")
        print()
    
    fig2, axes2 = plt.subplots(1, 2, figsize=(12, 5))
    
    cv = 0.2
    length_scale = 1.0
    
    E_field, Y_field, eigenvalues, eigenfunctions = generate_stiffness_field(
        x_points, mean_E=200e9, cv=cv, length_scale=length_scale, 
        n_terms=10, random_seed=42
    )
    
    axes2[0].plot(range(1, len(eigenvalues)+1), eigenvalues, 'bo-')
    axes2[0].set_xlabel('Serial number of eigenvalue')
    axes2[0].set_ylabel('Eigenvalue')
    axes2[0].set_title('Eigenvalue spectrum')
    axes2[0].grid(True, alpha=0.3)
    axes2[0].set_yscale('log')
    
    axes2[1].plot(x_points, eigenfunctions[:, 0], 'r-', label='Pattern 1')
    axes2[1].plot(x_points, eigenfunctions[:, 1], 'g-', label='Pattern 2')
    axes2[1].plot(x_points, eigenfunctions[:, 2], 'b-', label='Pattern 3')
    axes2[1].plot(x_points, eigenfunctions[:, 3], 'y-', label='Pattern 4')
    axes2[1].set_xlabel('Position (m)')
    axes2[1].set_ylabel('Characteristic Function value')
    axes2[1].set_title('First four Characteristic Function')
    axes2[1].grid(True, alpha=0.3)
    axes2[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    return True

if __name__ == "__main__":
    print("开始验证KL展开生成器...")
    success = validate_kl_generator()
    
    if success:
        print("KL展开生成器验证通过！")

        L = 5.0
        n_points = 50
        x_points = np.linspace(0, L, n_points)
        
        E_field, Y_field, eigenvalues, eigenfunctions = generate_stiffness_field(
            x_points, mean_E=200e9, cv=0.2, length_scale=1.0, 
            n_terms=10, random_seed=123
        )
        
        print(f"\n生成的刚度场样本统计：")
        print(f"  点数: {len(E_field)}")
        print(f"  均值: {np.mean(E_field)/1e9:.2f} GPa")
        print(f"  标准差: {np.std(E_field)/1e9:.2f} GPa")
        print(f"  变异系数: {np.std(E_field)/np.mean(E_field):.3f}")
        print(f"  最小值: {np.min(E_field)/1e9:.2f} GPa")
        print(f"  最大值: {np.max(E_field)/1e9:.2f} GPa")
        
        if np.all(E_field > 0):
            print("所有刚度值均为正（物理合理）")
        else:
            print("发现非正值，需要检查！")
    else:
        print("KL展开生成器验证失败")